In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import tempfile
import os
import tensorflow_model_optimization as tfmot

In [2]:
#加载数据集合
#加载MNIST数据集
(train_images, train_labels), (test_images, test_labels) = keras.datasets.mnist.load_data()
#将图像像素值规整到[0,1]
train_images = train_images / 255.0
test_images = test_images / 255.0
print("load data is ok")

load data is ok


In [6]:
model = keras.Sequential([ keras.layers.InputLayer(input_shape=(28, 28)),
                           keras.layers.Reshape(target_shape=(28, 28, 1)),
                           keras.layers.Conv2D(filters=12,kernel_size=(3, 3), activation='relu'),
                           keras.layers.MaxPooling2D(pool_size=(2,2)),
                           keras.layers.Flatten(),
                           keras.layers.Dense(10)])
model.compile(optimizer = 'adam',loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),metrics = ['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_1 (Reshape)          (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 26, 26, 12)        120       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 12)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2028)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                20290     
Total params: 20,410
Trainable params: 20,410
Non-trainable params: 0
_________________________________________________________________


In [7]:
model.fit(train_images,train_labels,epochs = 4,validation_split = 0.1)

_, pruned_keras_file = tempfile.mkstemp('.h5',dir='./')
tf.keras.models.save_model(model, pruned_keras_file, include_optimizer=False)
print('Saved pruned Keras model to:', pruned_keras_file)

Train on 54000 samples, validate on 6000 samples
Epoch 1/4
54000/54000 [==============================] - 6s 116us/sample - loss: 0.3169 - accuracy: 0.9100 - val_loss: 0.1354 - val_accuracy: 0.9643
Epoch 2/4
54000/54000 [==============================] - 6s 108us/sample - loss: 0.1278 - accuracy: 0.9637 - val_loss: 0.0879 - val_accuracy: 0.9787
Epoch 3/4
54000/54000 [==============================] - 6s 110us/sample - loss: 0.0885 - accuracy: 0.9747 - val_loss: 0.0738 - val_accuracy: 0.9803
Epoch 4/4
54000/54000 [==============================] - 6s 117us/sample - loss: 0.0714 - accuracy: 0.9791 - val_loss: 0.0666 - val_accuracy: 0.9832
Saved pruned Keras model to: D:\riky\jupyterpro\Tensorflow\tmp35qq2zf9.h5


# 对整个模型进行剪枝

In [14]:
#加载原来已经训练好的模型
base_model = keras.models.load_model('tmp35qq2zf9.h5')
base_model.summary()
print("******************************************************************")

#如果对整个模型进行剪枝，可以查看改模型的情况, 可以看到模型输出的名称都不一样
model_for_pruning = tfmot.sparsity.keras.prune_low_magnitude(base_model)
model_for_pruning.summary()

W0909 11:03:58.645002 10648 hdf5_format.py:177] No training configuration found in save file: the model was *not* compiled. Compile it manually.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_1 (Reshape)          (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 26, 26, 12)        120       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 12)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2028)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                20290     
Total params: 20,410
Trainable params: 20,410
Non-trainable params: 0
_________________________________________________________________
******************************************************************
Model: "sequential_1"
___________________________

# 修剪模型的某一部分(最好模型剪枝之前已经加载了权值)

In [25]:
#修剪模型的Dense Layer   ,可以通过网络的名字来进行确定剪枝层
def apply_pruning_to_dense(layer):
    print("layer name:",layer.name)
    if isinstance(layer,tf.keras.layers.Dense):
        print("Apply pruning to Dense")
        return tfmot.sparsity.keras.prune_low_magnitude(layer)
    return layer
#其中tf.keras.models.clone_model是对keras定义的层进行一些改变，具体看一看 官方api
model_for_pruning = tf.keras.models.clone_model(base_model,clone_function= apply_pruning_to_dense)
model_for_pruning.summary()

layer name: reshape_1
layer name: conv2d_1
layer name: max_pooling2d_1
layer name: flatten_1
layer name: dense_1
Apply pruning to Dense
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_1 (Reshape)          (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 26, 26, 12)        120       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 12)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2028)              0         
_________________________________________________________________
prune_low_magnitude_dense_1  (None, 10)                40572     
Total params: 40,692
Trainable params: 20,410
Non-trainable params: 20,282
_________________________________________

In [19]:
print(base_model.layers[0].name)  #查看模型某一层的名字

reshape_1


# 自定义剪枝操作，通过 tfmot.sparsity.keras.PrunableLayer 自定需要剪枝的参数，常有两种情况：（通常bia的prune会严重降低精度，默认是不会prune的，此处只作示例）

# Tensorboard可视化

In [32]:
base_model = keras.models.load_model('tmp35qq2zf9.h5')
# base_model.summary()

# #PolynomialDecay方法定义一个具有多项式衰减功能的修剪计划，也就是说修剪过程中的稀疏度是变化的，网络参数逐渐减少，稀疏度逐渐提高。
# pruning_schedule = tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
#                                                         final_sparsity=0.80,
#                                                         begin_step=0,
#                                                         end_step=end_step)
# #修改要在训练期间修剪的tf.keras层或模型，本例修剪的是整个模型的参数
# model_for_pruning = tfmot.sparsity.keras.prune_low_magnitude(model, pruning_schedule)

log_dir = tempfile.mkdtemp()
print(log_dir)#查看保存地址

model_for_pruning = tfmot.sparsity.keras.prune_low_magnitude(base_model)
model_for_pruning.summary()


callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),  #回调函数，使其在训练过程中处理修减更新
  tfmot.sparsity.keras.PruningSummaries(log_dir=log_dir), #提供用于跟踪进度和调试的日志
]


model_for_pruning.compile(loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
                          optimizer = "adam",
                          metrics = ["accuracy"])
model_for_pruning.fit(train_images, train_labels, callbacks = callbacks, epochs = 15)


W0909 11:54:45.299134 10648 hdf5_format.py:177] No training configuration found in save file: the model was *not* compiled. Compile it manually.


C:\Users\riky\AppData\Local\Temp\tmp_2xm4k78
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
prune_low_magnitude_reshape_ (None, 28, 28, 1)         1         
_________________________________________________________________
prune_low_magnitude_conv2d_1 (None, 26, 26, 12)        230       
_________________________________________________________________
prune_low_magnitude_max_pool (None, 13, 13, 12)        1         
_________________________________________________________________
prune_low_magnitude_flatten_ (None, 2028)              1         
_________________________________________________________________
prune_low_magnitude_dense_1  (None, 10)                40572     
Total params: 40,805
Trainable params: 20,410
Non-trainable params: 20,395
_________________________________________________________________
Train on 60000 samples
Epoch 1/15
60000/60000 [===================

# 可视化

In [35]:
tensorboard --logdir=log_dir  #就是使用tensorboard 打开改文件夹，要使用控制台才行

SyntaxError: can't assign to operator (<ipython-input-35-5341426ab8ac>, line 1)

# 使用strip_pruning去除之前的不可训练权重,并保存模型

In [39]:
#获得模型权重大小 
def get_gzipped_model_size(model):
    import os
    import zipfile
    _, keras_file = tempfile.mkstemp('.h5')
    model.save(keras_file, include_optimizer=False)
    
    _, zipped_file = tempfile.mkstemp('.zip')
    with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
        f.write(keras_file)
    return os.path.getsize(zipped_file)

In [40]:
#使用strip_pruning去除之前的不可训练权重,并保存模型
model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)#使用strip_pruning去除之前的不可训练权重,并保存模型
print("final model")
model_for_export.summary()

final model
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_1 (Reshape)          (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 26, 26, 12)        120       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 12)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2028)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                20290     
Total params: 20,410
Trainable params: 20,410
Non-trainable params: 0
_________________________________________________________________


In [42]:
print("\n")
print("Size of gzipped pruned model without stripping: %.2f bytes" % (get_gzipped_model_size(base_model)))
print("Size of gzipped pruned model with stripping: %.2f bytes" % (get_gzipped_model_size(model_for_export)))



Size of gzipped pruned model without stripping: 49131.00 bytes
Size of gzipped pruned model with stripping: 49131.00 bytes
